# "Analise casos de SRAG-COVID em crianças e adolecentes"
> "Dados dos casos de hospitalizações por SRAG do opendatasus"

- toc: true
- branch: master
- badges: false
- comments: false
- numbersections: true
- categories: [srag]
- image: images/some_folder/your_image.png
- hide:false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

## Objetivos
- Analisar comportamento dos casos e letalidade em crianças e adolecentes
- Analisar ao longo do tempo e em diferentes estados

In [1]:
#hide

import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from srag_functions import *

db_name = 'srag'
db_path = f'data/opendatasus/{db_name}.db'
conn = sql.connect(db_path)
df_srag = pd.read_sql(f'SELECT * FROM {db_name} WHERE ano >= 2020', conn)

In [2]:
#hide

def get_proportion_cases(df,index_cols,categories_cols):
    df_categories = pd.DataFrame(df.groupby(by=index_cols + categories_cols).size(),columns=['casos']).reset_index()
    df_categories = df_categories.pivot(index=index_cols,columns=categories_cols,values='casos')
    
    # ex: se index_cols = ['ano','SEM_PRI'], cada linha terá total por ano e semana
    df_subtotal = pd.DataFrame(df.groupby(by=index_cols).size(),columns=['total'])
    
    # ex: calcula proporção de cada categoria na "semana", entre 0.0 e 1.0
    df_rel = df_categories.div(df_subtotal.values,axis=0)
    
    # a princípio considera apenas a primeira categoria
    selected_category = categories_cols[0]
    
    df1 = pd.melt(df_rel,ignore_index=False,value_name='proporção').set_index(selected_category,append=True)
    df2 = pd.melt(df_categories,ignore_index=False,value_name='casos').set_index(selected_category,append=True)

    return pd.concat([df1,df2],axis=1).reset_index()


def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]
    

In [3]:
#hide

df_covid = df_srag.query('CLASSI_FIN == "COVID-19"')

## Proporção de casos de SRAG-COVID em crianças e adolecentes (até 19 anos)

In [4]:
#hide-input

index_cols = ['ano','SEM_PRI']
categories_cols =['faixa_etaria']

df_covid = df_srag.query('CLASSI_FIN == "COVID-19"')
df_casos_faixas = get_proportion_cases(df_covid,index_cols,categories_cols)
df_chart = df_casos_faixas.query('faixa_etaria == "00-20"')

alt.Chart(df_chart).mark_line(point=True).encode(
    x='SEM_PRI',
    y='proporção',
    color='ano:N',
    tooltip=['SEM_PRI','proporção','casos','ano']
)

alt.Chart(...)

## Total de casos de SRAG-COVID em crianças e adolecentes (até 19 anos)

In [5]:
#hide-input

alt.Chart(df_chart).mark_line(point=True).encode(
    x='SEM_PRI',
    y='casos',
    color='ano:N',
    tooltip=['SEM_PRI','proporção','casos','ano']
)

alt.Chart(...)

> Tabela com dados

In [6]:
#collapse-output
#hide-input

df_chart.style.apply(highlight_max,subset=['casos','proporção']).format({'casos':'{:.0f}','proporção':'{:.2%}'})

,ano,SEM_PRI,faixa_etaria,proporção,casos
0,2020,8,00-20,5.36%,3
1,2020,9,00-20,6.76%,5
2,2020,10,00-20,3.80%,10
3,2020,11,00-20,2.55%,29
4,2020,12,00-20,2.26%,82
5,2020,13,00-20,1.60%,86
6,2020,14,00-20,2.07%,144
7,2020,15,00-20,1.59%,157
8,2020,16,00-20,1.82%,255
9,2020,17,00-20,1.80%,328


## Proporção de óbitos de SRAG-COVID em crianças e adolecentes (até 19 anos)

In [7]:
#hide-input

index_cols = ['ano','SEM_PRI']
categories_cols =['faixa_etaria']

df_covid = df_srag.query('CLASSI_FIN == "COVID-19" and EVOLUCAO == "obito"')

df_obitos_faixas = get_proportion_cases(df_covid,index_cols,categories_cols).fillna(0)
df_chart = df_obitos_faixas.query('faixa_etaria == "00-20"')

alt.Chart(df_chart).mark_line(point=True).encode(
    x='SEM_PRI',
    y='proporção',
    color='ano:N',
    tooltip=['SEM_PRI','proporção','casos','ano']
)

alt.Chart(...)

## Total de óbitos de SRAG-COVID em crianças e adolecentes (até 19 anos)

In [8]:
#hide-input

alt.Chart(df_chart).mark_line(point=True).encode(
    x='SEM_PRI',
    y='casos',
    color='ano:N',
    tooltip=['SEM_PRI','proporção','casos','ano']
)

alt.Chart(...)

> Tabela com dados

In [9]:
#hide-input
#collapse-output

df_chart.style.apply(highlight_max,subset=['casos','proporção']).format({'casos':'{:.0f}','proporção':'{:.2%}'})

,ano,SEM_PRI,faixa_etaria,proporção,casos
0,2020,8,00-20,0.00%,0
1,2020,9,00-20,0.00%,0
2,2020,10,00-20,1.05%,1
3,2020,11,00-20,1.27%,4
4,2020,12,00-20,0.81%,8
5,2020,13,00-20,0.62%,10
6,2020,14,00-20,0.53%,13
7,2020,15,00-20,0.53%,19
8,2020,16,00-20,0.61%,33
9,2020,17,00-20,0.74%,54


## Letalidade dos casos de SRAG-COVID em crianças e adolecentes (até 19 anos)

> obs: Para o calculo da letalidade considerei tanto os casos de "obito" quanto "obito_outras_causas"

In [10]:
#hide-input

df_casos_concluidos = df_srag.query('CLASSI_FIN == "COVID-19" and EVOLUCAO in ("obito","cura","obito_outras_causas")')

index_cols = ['ano','SEM_PRI','faixa_etaria']

df_casos_concluidos_faixas = pd.DataFrame(df_casos_concluidos.groupby(by=index_cols).size(),columns=['casos'])

df_obitos = df_casos_concluidos.query('EVOLUCAO in ("obito","obito_outras_causas")')
df_obitos_faixas = pd.DataFrame(df_obitos.groupby(by=index_cols).size(),columns=['obitos'])

df_casos_concluidos_faixas
df_letalidate = pd.concat([df_casos_concluidos_faixas,df_obitos_faixas],axis=1).fillna(0) 
df_letalidate['letalidade'] = df_letalidate['obitos'] / df_letalidate['casos']


df_chart = df_letalidate.reset_index().query('faixa_etaria == "00-20"')
# outra forma de fazer a mesma coisa, não sei se seria mais eficiente.
# df_chart = df_letalidate.sort_index().loc[(slice(None),slice(None),slice('00-20')),:].reset_index()

alt.Chart(df_chart).mark_line(point=True).encode(
    x='SEM_PRI',
    y='letalidade',
    color='ano:N',
    tooltip=df_chart.columns.to_list()
)

alt.Chart(...)

> Tabela com dados

In [11]:
#collapse-output
#hide-input


df_chart.style.apply(highlight_max,subset=['casos','obitos','letalidade']).format({'obitos':'{:.0f}','letalidade':'{:.2%}'})

,ano,SEM_PRI,faixa_etaria,casos,obitos,letalidade
0,2020,8,00-20,3,0,0.00%
6,2020,9,00-20,3,0,0.00%
12,2020,10,00-20,9,1,11.11%
18,2020,11,00-20,24,4,16.67%
24,2020,12,00-20,70,8,11.43%
30,2020,13,00-20,81,10,12.35%
36,2020,14,00-20,133,14,10.53%
42,2020,15,00-20,138,19,13.77%
48,2020,16,00-20,236,34,14.41%
54,2020,17,00-20,302,55,18.21%


## Proporção de casos de SRAG-COVID em crianças e adolecentes (até 19 anos) por UF

In [12]:
#hide-input

index_cols = ['UF_RES']
categories_cols =['faixa_etaria']

df_covid = df_srag.query('CLASSI_FIN == "COVID-19"')
df = df_covid.query('UF_RES != "nan_nd"')#in ("29_Bahia","33_Rio de Janeiro","35_São Paulo")')
# df = df.query('SEM_PRI_ABS >= 15 and SEM_PRI_ABS <= 69')
df_casos_faixas = get_proportion_cases(df,index_cols,categories_cols)
df_chart = df_casos_faixas.query('faixa_etaria == "00-20"')


category = 'UF_RES'
chart = alt.Chart(df_chart).mark_bar().encode(
#     x='UF_RES',
    y='proporção:Q',
    x=alt.X('UF_RES:N',sort='y'),
    color= category,
    tooltip=df_chart.columns.to_list()
)

ns_opacity = 0.01
selection = alt.selection_multi(empty='all', fields=[category], bind='legend')
chart = chart.add_selection(
    selection
).encode(
    opacity=alt.condition(selection, alt.value(1.0), alt.value(ns_opacity))
)
chart

alt.Chart(...)

## Proporção de óbitos por SRAG-COVID em crianças e adolecentes (até 19 anos) por UF

In [13]:
#hide-input

index_cols = ['UF_RES']
categories_cols =['faixa_etaria']

df_covid = df_srag.query('CLASSI_FIN == "COVID-19"')
df = df_covid.query('UF_RES != "nan_nd"')#in ("29_Bahia","33_Rio de Janeiro","35_São Paulo")')
# df = df.query('SEM_PRI_ABS >= 15 and SEM_PRI_ABS <= 69')
df = df.query('EVOLUCAO == "obito"')
df_casos_faixas = get_proportion_cases(df,index_cols,categories_cols)
df_chart = df_casos_faixas.query('faixa_etaria == "00-20"')


category = 'UF_RES'
chart = alt.Chart(df_chart).mark_bar().encode(
#     x='UF_RES',
    y='proporção:Q',
    x=alt.X('UF_RES:N',sort='y'),
    color= category,
    tooltip=df_chart.columns.to_list()
)

ns_opacity = 0.01
selection = alt.selection_multi(empty='all', fields=[category], bind='legend')
chart = chart.add_selection(
    selection
).encode(
    opacity=alt.condition(selection, alt.value(1.0), alt.value(ns_opacity))
)
chart

alt.Chart(...)

> Análise por semana - diferentes UF

In [14]:
#hide-input

index_cols = ['UF_RES','SEM_PRI_ABS']
categories_cols =['faixa_etaria']

df_covid = df_srag.query('CLASSI_FIN == "COVID-19"')
df = df_covid.query('UF_RES != "nan_nd"')#in ("29_Bahia","33_Rio de Janeiro","35_São Paulo")')
df = df.query('SEM_PRI_ABS >= 15 and SEM_PRI_ABS <= 69')
df_casos_faixas = get_proportion_cases(df,index_cols,categories_cols)
df_chart = df_casos_faixas.query('faixa_etaria == "00-20"')

category = 'UF_RES'
chart = alt.Chart(df_chart).mark_line(point=True).encode(
    x='SEM_PRI_ABS',
    y='proporção',
    color= category,
    tooltip=df_chart.columns.to_list()
)

ns_opacity = 0.01
selection = alt.selection_multi(empty='all', fields=[category], bind='legend')
chart = chart.add_selection(
    selection
).encode(
    opacity=alt.condition(selection, alt.value(1.0), alt.value(ns_opacity))
)
chart

alt.Chart(...)

> Análise por semana - seleciona 1 UF

In [15]:
#hide-input

chart = chart.transform_filter(
    selection
)
chart

alt.Chart(...)